In [16]:
# openssl rsa -text < CTF_Challenge/picoctf/corrupt-key-1/private.key
def small_roots(f, X, beta=1.0, m=None):
    N = f.parent().characteristic()
    delta = f.degree()
    if m is None:
        epsilon = RR(beta^2/f.degree() - log(2*X, N))
        m = max(beta**2/(delta * epsilon), 7*beta/delta).ceil()
    t = int((delta*m*(1/beta - 1)).floor())
    
    f = f.monic().change_ring(ZZ)
    P,(x,) = f.parent().objgens()
    g  = [x**j * N**(m-i) * f**i for i in range(m) for j in range(delta)]
    g.extend([x**i * f**m for i in range(t)]) 
    B = Matrix(ZZ, len(g), delta*m + max(delta,t))

    for i in range(B.nrows()):
        for j in range(g[i].degree()+1):
            B[i,j] = g[i][j]*X**j

    B =  B.LLL()
    f = sum([ZZ(B[0,i]//X**i)*x**i for i in range(B.ncols())])
    roots = set([f.base_ring()(r) for r,m in f.roots() if abs(r) <= X])
    return [root for root in roots if N.gcd(ZZ(f(root))) >= N**beta]

def recover(p_high, n, m):
        p_bits = (len(bin(n))-2)//2
        p_high_bits = len(bin(p_high)) - 2
        PR.<x> = PolynomialRing(Zmod(n))
        f = p_high * 2**(p_bits-p_high_bits) + x
        x = small_roots(f, X=2**(p_bits-p_high_bits), beta=0.4, m=m)
        if x == []:
                return None
        p = int(f(x[0]))
        return p

def solve(_p_high, bits, m):
    for x in range(2**bits, -1, -1):
        _p = _p_high + x * 2**(256-bits)
        p_high = int(bin(_p)[:256+bits+2], 2)
        p = recover(p_high, n, m)
        if p is not None:
            return p

mod = """00:b8:cb:1c:ca:99:b6:ac:41:87:6c:18:84:57:32:a5:cb:fc:87:5d:f3:46:ee:90:02:ce:60:85:08:b5:fc:f6:b6:0a:5a:c7:72:2a:2d:64:ef:74:e1:44:3a:33:8e:70:a7:3e:63:a3:03:f3:ac:9a:df:19:85:95:69:9f:6e:9f:30:c0:09:d2:19:c7:d9:8c:4e:c8:42:03:61:08:34:02:9c:79:56:7e:fc:08:f6:6b:4b:c3:f5:64:bf:b5:71:54:6a:06:b7:e4:8f:b3:5b:b9:cc:ea:9a:2c:d4:43:49:f8:29:24:20:78:df:a6:4d:52:59:27:bf:d5:5d:09:9c:02:4f"""
mod = "".join(mod.split(":"))
N = int(mod, 16)
e = 65537
p = "00:e7:00:56:8f:f5:06:bd:58:92:af:92:59:21:25:e0:6c:be:9b:d4:5d:fe:af:e9:31:a3:33:c1:34:63:02:3d:4f:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00:00"
p = "".join(p.split(":"))
p_high = int(p, 16)

p = solve(p_high, 7, 18)
print(p)

12098520864598198757294135341465388062087431109285224283440314414683283061468500249596026217234382854875647811812632201834942205849073893715844547051090363
